# Producer :

In [ ]:
from confluent_kafka import Producer
import socket

# Configuration du Producer avec compression
conf = {
    'bootstrap.servers': 'localhost:9092',
    'client.id': socket.gethostname(),
    'compression.type': 'gzip',  # Compression type: gzip, snappy, lz4, or zstd
    'message.max.bytes': 500000000  # Increase if necessary
}
producer = Producer(conf)

# Fonction pour charger un fichier en bytes
def send_file(file_path, topic):
    try:
        with open(file_path, 'rb') as file:
            file_bytes = file.read()  # Lire le fichier en bytes
            
            # Vérifier si la taille dépasse les limites du message
            if len(file_bytes) > conf.get('message.max.bytes', 1048576):  # 1 MB par défaut
                raise ValueError("File size exceeds Kafka message size limit even with compression.")
            
            producer.produce(topic,key=file_path[6:], value=file_bytes)
            print(f"File {file_path} sent successfully to topic {topic}")
    except Exception as e:
        print(f"Failed to send file: {e}")
    finally:
        producer.flush()

# Envoi d'une image compressée automatiquement par Kafka
send_file('envoi/bikes.mp4', 'files_topic')

File Images/1.png sent successfully to topic files_topic


# Consummer :

In [2]:
from confluent_kafka import Consumer

# Configuration du Consumer
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'file_group',
    'auto.offset.reset': 'earliest'
}
consumer = Consumer(conf)

# Souscription au topic
topic = 'files_topic'
consumer.subscribe([topic])

# Fonction pour recevoir et sauvegarder un fichier
def receive_file():
    try:
        while True:
            msg = consumer.poll(1.0)
            if msg is None:
                continue
            if msg.error():
                print(f"Consumer error: {msg.error()}")
                continue
            output_path = 'recu/'+msg.key().decode('utf-8')
            # Sauvegarder les bytes reçus comme un fichier
            with open(output_path, 'wb') as file:
                file.write(msg.value())
                print(f"File received and saved to {output_path}")
    except KeyboardInterrupt:
        print("Consumer interrupted.")
    finally:
        consumer.close()

# Réception de l'image
receive_file()

File received and saved to images_recu/bikes.mp4
Consumer interrupted.


# Dedection

In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import time
 
# cap = cv2.VideoCapture(0)  # For Webcam
#cap.set(3, 1280)
#cap.set(4, 720)
cap = cv2.VideoCapture("images_recu/bikes.mp4")  # For Video
 
 
model = YOLO("yolov8n.pt")

classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]
 
prev_frame_time = 0
new_frame_time = 0
 
while True:
    new_frame_time = time.time()
    success, img = cap.read()
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
            cvzone.cornerRect(img, (x1, y1, w, h))
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
 
            cvzone.putTextRect(img, f'{classNames[cls]} {conf}', (max(0, x1), max(35, y1)), scale=1, thickness=1)
 
    fps = 1 / (new_frame_time - prev_frame_time)
    prev_frame_time = new_frame_time
    print(fps)
 
    cv2.imshow("Image", img)
    cv2.waitKey(1)

In [1]:
from ultralytics import YOLO
import cv2
 
model = YOLO('yolov8n.pt')
results = model("images_recu/1.png", show=True)
cv2.waitKey(0)

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\HP\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

image 1/1 c:\Users\HP\Desktop\Project_2-_kafka\images_recu\1.png: 448x640 5 persons, 9 motorcycles, 123.1ms
Speed: 5.0ms preprocess, 123.1ms inference, 3.0ms postprocess per image at shape (1, 3, 448, 640)


-1